In [1]:
%matplotlib inline
"""
Exemple de Cifar10 avec PyTorch
Exemple d'apprentissage pas transfert avec ResNet18
Version avec GPU
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn.functional as F
fonction_cout = F.cross_entropy

def taux_bonnes_predictions(lot_Y_predictions, lot_Y):
    predictions_categorie = torch.argmax(lot_Y_predictions, dim=1)
    return (predictions_categorie == lot_Y).float().mean()

import torchvision
from torchvision import datasets, models, transforms

modele = models.alexnet(pretrained=True)

# Désactiver l'apprentissage des paramètres pour réutiliser les valeurs déjà entraînées
for parametre in modele.parameters():
    parametre.requires_grad = False        

# Modifier la dernière couche pour nouvelles données
nombre_classes = 10

from torch import nn
modele.classifier[6] = nn.Linear(4096,nombre_classes)

pretraitement = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor(),
transforms.Normalize(
mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225]
)])

#Chargement des données
ds = torchvision.datasets.CIFAR10(root = "./data", train = True, download = True, transform = pretraitement)
ds_ent, ds_valid = torch.utils.data.random_split(ds, [40000, 10000])

#Création du DataLoader avec le dataset
dl_ent = torch.utils.data.DataLoader(ds_ent, batch_size=100, shuffle = True)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=100)

print("Parametres à entraîner:")
parametres_a_entrainer = []
for nom,parametre in modele.named_parameters():
    if parametre.requires_grad == True:
        parametres_a_entrainer.append(parametre)
        print("\t",nom)

from torch import optim
optimiseur = optim.SGD(parametres_a_entrainer, lr=0.001,momentum=0.9)

# Déterminer si un GPU est disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Placer le modèle en mode GPU si possible
modele = modele.to(device)
print('Entrainement sur ',device)

import time
def entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10):

    debut = time.time()

    # Listes pour les métriques par epoch
    liste_cout_moyen_ent = []
    liste_taux_moyen_ent = []
    liste_cout_moyen_valid = []
    liste_taux_moyen_valid = []
    
    # Boucle d'apprentissage
    for epoch in range(nb_epochs):
        cout_total_ent = 0 # pour cumuler les couts par mini-lot
        taux_bonnes_predictions_ent = 0 # pour cumuler les taux par mini-lot
        modele.train() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        
        # Boucle d'apprentissage par mini-lot pour une epoch
        for lot_X, lot_Y in dl_ent:
            
            # Les données doivent être placés en GPU pour être compatibles avec le modèle
            lot_X = lot_X.to(device)
            lot_Y = lot_Y.to(device)

            optimiseur.zero_grad() # Remettre les dérivées à zéro
            lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
            cout = fonction_cout(lot_Y_predictions, lot_Y)
            cout.backward() # Calcul des gradiants par rétropropagation
            with torch.no_grad():
                cout_total_ent +=cout
                taux_bonnes_predictions_ent += taux_bonnes_predictions(lot_Y_predictions, lot_Y)
            optimiseur.step() # Mise à jour des paramètres
            # scheduler.step()
        # Calculer les moyennes par mini-lot
        with torch.no_grad():
            cout_moyen_ent = cout_total_ent/len(dl_ent)
            taux_moyen_ent = taux_bonnes_predictions_ent/len(dl_ent)
       
        modele.eval() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        with torch.no_grad():
            # Les données de validation doivent aussi être placés en GPU
            cout_valid = sum(fonction_cout(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
            taux_bons_valid = sum(taux_bonnes_predictions(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
        cout_moyen_valid = cout_valid/len(dl_valid)
        taux_moyen_valid = taux_bons_valid/len(dl_valid)
        print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  taux moyen entraînement = {taux_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  coût moyen validation = {cout_moyen_valid}')
        print(f'-------- > epoch {epoch+1}:  taux moyen validation = {taux_moyen_valid}')
    
        liste_cout_moyen_ent.append(cout_moyen_ent)
        liste_taux_moyen_ent.append(taux_moyen_ent)
        liste_cout_moyen_valid.append(cout_moyen_valid)
        liste_taux_moyen_valid.append(taux_moyen_valid)
        
        temps_ecoule = time.time() - debut
        print('Temps écoulé : {:.0f}m {:.0f}s'.format(temps_ecoule // 60, temps_ecoule % 60))

    
    # Affichage du graphique d'évolution des métriques par epoch
    import numpy as np
    import matplotlib.pyplot as plt
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_ent,label='Erreur entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_valid,label='Erreur validation')
    plt.title("Evolution du coût")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='upper center')
    plt.show()
        
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_ent,label='Taux bonnes réponses entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_valid,label='Taux bonnes réponses validation')
    plt.title("Evolution du taux")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='center')
    plt.show()

entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\Robert/.cache\torch\checkpoints\alexnet-owt-4df8aa71.pth



Files already downloaded and verified
Parametres à entraîner:
	 classifier.6.weight
	 classifier.6.bias
Entrainement sur  cpu


KeyboardInterrupt: 